# Vorbereitungen

In [ ]:
!pip install openai pydantic langchain langchain-openai

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


In [ ]:
from openai import OpenAI
from typing import Any, Type
from langchain_core.tools import BaseTool
import json
import requests
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

## "Function-Calling"

Damit wir eine Funktion aufrufen können, müssen wir diese definieren. Wichtig dabei ist:
- Type-Annotations: wir müssen die Typen der In- und Output Variablen definieren
- Wir müssen die Funktion per Docstring beschreiben - mit dieser Beschreibung helfen wir dem LLM zu erkennen, wofür die Funktion verwendet werden kann 

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers together.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

LangChain bietet uns mit "convert_to_openai_tool" einen Helfer, der die Funktionsdefinition automatisch erzeugt. Wir müssen diese also nicht mehr kompliziert selbst schreiben.

In [ ]:
print(json.dumps(convert_to_openai_tool(multiply), indent=2))

Wir können die funktion dann dem LLM "bekannt" machen und einen Prompt abschicken der die Funktion benötigt.

In [ ]:
tools = [convert_to_openai_tool(multiply)]
llm_with_tools = llm.bind_tools(tools)
llm_with_tools.invoke("what's 3 * 12")

LangChain bietet uns noch einen weiteres Tool, nämlich den `JsonOutputToolsParser` - damit wird die aufzurufende Funktion und die Parameter extrahiert.

In [ ]:
tool_chain = llm_with_tools | JsonOutputToolsParser()
result = tool_chain.invoke("what's 3 * 12")
result

Jetzt können wir die Funktion aufrufen und das Ergebnis der Multiplikation sehen

In [ ]:
multiply(**result[0]['args'])

## APIs aufrufen mit "Function Calling"

In [ ]:
from langchain.schema import AIMessage, HumanMessage, FunctionMessage

In [ ]:
def product_search(keyword: str) -> str:
    """Search product that contains a keyword
    returns a json string containing a list of products with product details

    Args:
        keyword: Keyword to search for
    """
    response = requests.get(f'https://dummyjson.com/products/search?q={keyword}')
    return response.text

In [ ]:
llm_with_tools = llm.bind_tools([convert_to_openai_tool(product_search)])
tool_chain = llm_with_tools | JsonOutputToolsParser()

In [ ]:
messages = [HumanMessage(content="Ich brauche ein neues iPhone, welches hat die beste Bewertung?")]
result = tool_chain.invoke(messages)

In [ ]:
search_result = product_search(**result[0]['args'])
json.loads(search_result)

In [ ]:
messages.append(FunctionMessage(name='product_search', content=search_result))
result = llm.invoke(messages)

In [ ]:
print(result.content)